In [1]:
import pandas as pd
import os

from util.elmer_helpers import read_from_elmer_geo
from census_api import CensusApi


api_key = os.getenv('CENSUS_KEY','no variable found')
print(f'API Key: {api_key}')

county_ids = [53033,53035,53053,53061]
state_id = 53

c = CensusApi(api_key)

API Key: 1b373fe94cdee2c96c20b3c1f14ed6b1d6b92562


In [2]:
elmer_cities_gdf = read_from_elmer_geo('CITIES', ['city_name','city_fips','cnty_fips','rgeo_class','class_desc'])

c:\Users\JKolberg\PythonProjects\control_totals\.venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [6]:
def create_cnty_place_id(df):
    return ('53' + df['cnty_fips'].astype(int).astype(str).str.zfill(3) + df['city_fips'].astype(int).astype(str).str.zfill(5)).astype('int64')

elmer_cities_gdf['cnty_place_id'] = create_cnty_place_id(elmer_cities_gdf)

In [10]:
blk = read_from_elmer_geo('BLOCK2020', ['geoid20'])

c:\Users\JKolberg\PythonProjects\control_totals\.venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
# get ofm data for cities
# get decennial census data for cities
# get growth targets for cities

In [16]:
import pyodbc
import pandas as pd

def read_from_elmer(table_name, columns):
    conn_string = "DRIVER={ODBC Driver 17 for SQL Server}; SERVER=SQLserver; DATABASE=Elmer; trusted_connection=yes"
    sql_conn = pyodbc.connect(conn_string)
    cols_str = ', '.join(columns)
    sql_query = f'select {cols_str} from {table_name}'
    df = pd.read_sql(sql=sql_query, con=sql_conn)
    return df

ofm2020 = read_from_elmer('ofm.v_estimates_2020',['*'])

C:\Users\JKolberg\AppData\Local\Temp\ipykernel_20500\1247252603.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sql_query, con=sql_conn)


In [18]:
ofm2019 = read_from_elmer('ofm.v_estimates_2019',['*'])

C:\Users\JKolberg\AppData\Local\Temp\ipykernel_20500\1247252603.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sql_query, con=sql_conn)


In [24]:
reg = read_from_elmer_geo('REGIONAL_GEOGRAPHIES', ['city_name','cnty_name','juris','feat_type','rgeo_class','class_desc'])

c:\Users\JKolberg\PythonProjects\control_totals\.venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [22]:
reg.explore('cnty_name',categorical=True).save('reg.html')

In [25]:
reg

,city_name,cnty_name,juris,feat_type,rgeo_class,class_desc,geometry
0,Algona,King,Algona,city,4.0,CitiesTowns,"POLYGON ((1291988.75 102374.367, 1291988.875 1..."
1,Auburn,King,Auburn,city,2.0,Core,"POLYGON ((1292142.875 132236.578, 1292024.321 ..."
2,None,King,Auburn PAA,uninc,5.0,UU,"MULTIPOLYGON (((1294872.625 105982.445, 129487..."
3,None,King,Bear Creek UPD,uninc,5.0,UU,"POLYGON ((1346154.471 266557.317, 1344996.808 ..."
4,Beaux Arts,King,Beaux Arts,city,4.0,CitiesTowns,"POLYGON ((1302122 217033.531, 1301744.25 21703..."
...,...,...,...,...,...,...,...
158,None,Snohomish,Stanwood UGA,uninc,5.0,UU,"MULTIPOLYGON (((1273132.795 463193.316, 127313..."
159,Sultan,Snohomish,Sultan,city,4.0,CitiesTowns,"POLYGON ((1401491.617 324480.996, 1402142.769 ..."
160,None,Snohomish,Sultan UGA,uninc,5.0,UU,"MULTIPOLYGON (((1408705.649 322172.03, 1408720..."
161,Woodway,Snohomish,Woodway,city,4.0,CitiesTowns,"POLYGON ((1256708.625 296915.812, 1256691.878 ..."


In [10]:
dec_cols_dict = {
    'total_population':['P1_001N'],
    'housing_units':['H1_001N'],
    'households':['H1_002N'],
    'group_quarters':['P5_001N']
}

In [11]:
dec = (
    c.get_dec_data(dec_cols_dict, 2020, 'block', 'pl', county_ids,state_id)
)

In [ ]:
dec = (
    dec.assign()
)

,geoid,name,total_population,housing_units,households,group_quarters
0,530330001011000,"Block 1000, Block Group 1, Census Tract 1.01, ...",74.0,20.0,20.0,0.0
1,530330001011001,"Block 1001, Block Group 1, Census Tract 1.01, ...",99.0,30.0,25.0,0.0
2,530330001011002,"Block 1002, Block Group 1, Census Tract 1.01, ...",289.0,246.0,220.0,0.0
3,530330001011004,"Block 1004, Block Group 1, Census Tract 1.01, ...",54.0,18.0,1.0,5.0
4,530330001011005,"Block 1005, Block Group 1, Census Tract 1.01, ...",649.0,443.0,411.0,0.0
...,...,...,...,...,...,...
54481,530619900020055,"Block 0055, Block Group 0, Census Tract 9900.0...",0.0,0.0,0.0,0.0
54482,530619900020058,"Block 0058, Block Group 0, Census Tract 9900.0...",0.0,0.0,0.0,0.0
54483,530619900020061,"Block 0061, Block Group 0, Census Tract 9900.0...",0.0,0.0,0.0,0.0
54484,530619901000001,"Block 0001, Block Group 0, Census Tract 9901, ...",0.0,0.0,0.0,0.0


In [21]:
dec = (
    pd.DataFrame(c.pl.state_place(['NAME','P1_001N','H1_001N','H1_002N','P5_001N'], state_id, '*'))
    .rename(columns=dec_cols_dict)
    .assign(
        place_id = lambda x: (x['state'].astype(str) + x['place'].astype(str)).astype('int64'),
        cdp = lambda x: x['NAME'].str.contains('CDP')
    )
    .query('cdp == False')
    .drop(columns=['state','place','cdp'])
    .set_index(['place_id','NAME'])
    # set all columns to int
    .astype('int64')
    .reset_index()
)

In [26]:
df = elmer_cities.merge(dec, on='place_id', how='outer')

In [27]:
df.to_csv('cities_test.csv', index=False)